In [1]:
!pip install InstructorEmbedding
!pip install sentence_transformers
!/opt/bin/nvidia-smi

Wed Oct  4 12:37:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
# print(torch.__version__)
# print(torch.version.cuda)
# print(torch.backends.cudnn.version())
# print(torch.cuda.get_device_name(0))
# print(torch.cuda.is_available())

In [3]:
# # #Uninstall the current CUDA version
# !apt-get --purge remove cuda nvidia* libnvidia-*
# !dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
# !apt-get remove cuda-*
# !apt autoremove
# !apt-get update

# #Download CUDA 10.0
# !wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.2.89-1_amd64.deb
# #install CUDA kit dpkg
# !yes | dpkg -i cuda-repo-ubuntu1804_10.2.89-1_amd64.deb
# !sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
# !apt-get update
# !apt-get install cuda-10-2

In [4]:
!nvcc --version
!cat /usr/local/cuda/version.txt
!cat /usr/include/x86_64-linux-gnu/cudnn_v*.h | grep CUDNN_MAJOR -A 2

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
cat: /usr/local/cuda/version.txt: No such file or directory
#define CUDNN_MAJOR 8
#define CUDNN_MINOR 9
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


In [5]:
from google.colab import drive
from InstructorEmbedding import INSTRUCTOR
drive.mount('/mntDrive')
import os
import json
import numpy as np

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [6]:
def get_passage_embds(model, date, passages_dir_prefix, para_embs_dir_prefix):
    model_name = model.split('/')[-1]
    passage_dir = passages_dir_prefix.format(date)
    para_embs_dir = para_embs_dir_prefix.format(model_name, date)

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = INSTRUCTOR(model, device=device)
    instruction = "Represent the Financial paragraph for retrieval: "

    for doc in os.listdir(passage_dir):
        print(doc)
        if os.path.exists(os.path.join(para_embs_dir, doc)):
          continue
        doc_fp = os.path.join(passage_dir, doc)
        save_fp = os.path.join(para_embs_dir, doc)
        if not os.path.exists(para_embs_dir):
          os.makedirs(para_embs_dir)
        with open(doc_fp, "r") as f:
            passages = json.load(f)

        ptxt_list = []
        pctxt_list = []
        for pid in range(len(passages["passages_text"])):
            ptxt_list.append(passages["passages_text"][pid]["ptxt"])
            if "passages_composite_text" in passages:
                pctxt_list.append(passages["passages_composite_text"][pid]["pctxt"])

        ptxt_embs = []
        pctxt_embs = []

        for text in ptxt_list:
          embs = model.encode([[instruction, text]]).tolist()
          ptxt_embs.append(embs[0])
        for text in pctxt_list:
          embs = model.encode([[instruction, text]]).tolist()
          pctxt_embs.append(embs[0])

        with open(os.path.join(para_embs_dir, doc), "w") as f:
            json.dump({"ptxt_embs": ptxt_embs, "pctxt_embs": pctxt_embs}, f)

In [7]:
def query_embs(model, date, groundtruth_fp, query_emb_prefix):
    model_name = model.split('/')[-1]
    query_embs = query_emb_prefix.format(model_name, date)

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = INSTRUCTOR(model, device=device)
    instruction = "Represent the Financial query for retrieval: "

    with open(groundtruth_fp, "r") as f:
        groundtruth = json.load(f)

        query_list = []
        for doc in groundtruth:
            for question in doc["questions"]:
                # encode query
                embs = model.encode([[instruction, question["query"]]]).tolist()
                question["qemb"] = embs[0]

    with open(query_embs, "w") as f:
        json.dump(groundtruth, f)

In [8]:
# from InstructorEmbedding import INSTRUCTOR
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model1 = 'hkunlp/instructor-large'
# model2 = 'hkunlp/instructor-xl'
# for model_name in [model1, model2]:
#   model = INSTRUCTOR(model_name, device=device)
#   total_params = sum(p.numel() for p in model.parameters())
#   print(model_name, ": ", total_params)

In [9]:
data_p =  '/mntDrive/MyDrive/Colab Notebooks/'

date = "0708"

passages_dir_prefix = data_p + "data/groundtruth/passages_{0}"
para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_embs_{1}"

# passages_dir_prefix = data_p + "data/groundtruth/passages_proc_table_{0}"
# para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_proc_table_embs_{1}"

# passages_dir_prefix = data_p + "data/groundtruth/passages_chunk_{0}"
# para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_embs_chunk_{1}"

# passages_dir_prefix = data_p + "data/groundtruth/passages_table2text_tid_{0}"
# para_embs_dir_prefix = data_p + "data/groundtruth/{0}/para_embs_table2text_{1}"

groundtruth_fp = data_p +  "data/groundtruth/qa_groundtruth.json"
query_emb_prefix =  data_p + "data/groundtruth/{0}/query_emb_{1}.json"

model1 = 'hkunlp/instructor-large'
model2 = 'hkunlp/instructor-xl'

get_passage_embds(model2, date, passages_dir_prefix, para_embs_dir_prefix)
# query_embs(model2, date, groundtruth_fp, query_emb_prefix)

load INSTRUCTOR_Transformer
max_seq_length  512
280cb2e157d1e8b4586809fb14861f04.json
6acd4a68d18f349e3773e5b8369479d1.json
f61a41683f7f540de066fccee869deee.json
29a89d83a0f88569882adf0c969d385c.json
599d4ef0e5c755af465c0a9941bffb95.json
4d41ea7a63b2d9b5cc3cb24ca6c7e9ac.json
bf53025254bf57a1a13b4a3c7cc841c1.json
6a6cdcc9aa172495e08204d4893ee018.json
a62c00f8a92864a6e198b67593066a9d.json
13c5057e813cedb2666f3422749a2070.json
5a65b0e6a4ab79f96a4d3e05674d8462.json
4f00cc063cb8e99a038d7b4a2f77627b.json
87d7fddb42cdd7554e61be3c70ebb97b.json
30e46d073b61081f322200a8a8502b41.json
6f99a700db4e43c1bc6abf015c2b677a.json
f6368ece2c45f8080d07ccc5817d2431.json
0ba33bc0610e557810de948f4248719e.json
3fa3766530d2f9f7c533e997f5d7a5dd.json
028ee763c1f90a74df4736ce7b5ffba4.json
5f9e7b09281aad0e829b4ac500bbc419.json
f1068e3989914809aef217d162db5f2f.json
525032451ab45a3ba6d2629c06d68d1d.json
82f59cefbb50bb0f153ca236d85424db.json
ecbc9696600502b3dc792c59233f5efe.json
8f9110a2af02383fbae1dd8502df63a0.json
2f